<a href="https://colab.research.google.com/github/AmirGhnbr/Tweet-classification-tensorflow/blob/dev-branch/Tweet_classification_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This project aims to classify a particular tweet is related to disaster or not.

## make sure using GPU

In [1]:
!nvidia-smi

Mon Oct 10 20:47:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# get the helper function
!wget https://raw.githubusercontent.com/AmirGhnbr/Tweet-classification-tensorflow/main/helper.py

--2022-10-10 20:47:35--  https://raw.githubusercontent.com/AmirGhnbr/Tweet-classification-tensorflow/main/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10566 (10K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]  10.32K  --.-KB/s    in 0s      

2022-10-10 20:47:35 (72.4 MB/s) - ‘helper.py’ saved [10566/10566]



In [3]:
from helper import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Download text dataset from [Kaggle](https://www.kaggle.com/c/nlp-getting-started/data)

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2022-10-10 20:47:39--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 142.250.81.208, 172.217.15.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2022-10-10 20:47:39 (86.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(5)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# Lets see some random tweets
import random
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
Tsunami - DVBBS &amp; Borgeous (Arceen Festival Trap Remix) https://t.co/743JoqazrT via @YouTube

---

Target: 0 (not real disaster)
Text:
Then the stylist who'd been silent says 'there's a cool show on Amazon Prime called Catastrophe...'

---

Target: 1 (real disaster)
Text:
11 soldiers killed in ISIS suicide bomb in air base east Homs SYRIA NEWS | ZAMAN ALWSL  - http://t.co/V8juC5eK1A  #ISIS #Homs

---

Target: 1 (real disaster)
Text:
'It looks like a mudslide!' And #GBBO is back with a belter!

---

Target: 1 (real disaster)
Text:
Be careful during hurricane season ???? https://t.co/bFtOU2nybW

---



## Split the data

In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels  = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                        train_df_shuffled["target"].to_numpy(),
                                                                        test_size=0.1, # dedicate 10% of samples to validation set
                                                                        random_state=42)


In [11]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [12]:
train_sentences[:10], train_labels[:10] 

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [13]:
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_mode='int',
                                    output_sequence_length=15)
text_vectorizer

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
sample_sen = "this is sample sentence"
text_vectorizer([sample_sen])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    9, 8839,    1,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

## Baseline model

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfid", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfid', TfidfVectorizer()), ('clf', MultinomialNB())])

In [17]:
baseline_score = model_0.score(val_sentences, val_labels)
baseline_score

0.7926509186351706

## Model With LSTM

In [18]:
import tensorflow.keras.layers as layers
tf.random.set_seed(42)

model_1_embedding = tf.keras.layers.Embedding(input_dim=10000,
                                              output_dim=128,
                                              embeddings_initializer="uniform",
                                              input_length=15)

inputs= tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_1_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.models.Model(inputs, output)
model_1.summary()

(None, 15, 128)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
___________________________________

In [19]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [20]:
model_1_hist = model_1.fit(train_sentences, train_labels, epochs=5,
            validation_data= (val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 10s 8ms/step - loss: 0.5100 - accuracy: 0.7416 - val_loss: 0.4566 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3176 - accuracy: 0.8717 - val_loss: 0.5138 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2201 - accuracy: 0.9152 - val_loss: 0.5858 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1556 - accuracy: 0.9428 - val_loss: 0.6041 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1076 - accuracy: 0.9594 - val_loss: 0.8746 - val_accuracy: 0.7507


In [21]:
model_1_preds = model_1.predict(val_sentences)
model_1_preds[:10], model_1_preds.shape

(array([[0.007126  ],
        [0.78736776],
        [0.9996376 ],
        [0.05679169],
        [0.0025822 ],
        [0.9996238 ],
        [0.92170197],
        [0.9997993 ],
        [0.9994954 ],
        [0.66457367]], dtype=float32), (762, 1))

In [22]:
model1_preds = tf.squeeze(tf.round(model_1_preds))
model1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [23]:
val_sentences[-5:], val_labels[-5:], model1_preds[-5:].numpy()

(array(["That's the ultimate road to destruction",
        '@SetZorah dad why dont you claim me that mean that not right we look the same same eyes same blood same xbox 360 SMH -.-',
        'FedEx will no longer transport bioterror pathogens in wake of anthrax lab mishaps http://t.co/lHpgxc4b8J',
        'Crack in the path where I wiped out this morning during beach run. Surface wounds on left elbow and right knee. http://t.co/yaqRSximph',
        'I liked a @YouTube video from @dannyonpc http://t.co/AAuIzGGc9Q Battlefield Hardline - 11 NEW WEAPONS - New map - Throwingknifes!'],
       dtype=object),
 array([0, 0, 0, 0, 0]),
 array([1., 1., 1., 0., 0.], dtype=float32))

## Model 2 using gru

In [24]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim = 10000,
                                     output_dim = 128,
                                     name="embedding_2")

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.models.Model(inputs, outputs)
model_2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________

In [25]:
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [26]:
model_2_hist = model_2.fit(train_sentences, train_labels, epochs=5,
            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.5242 - accuracy: 0.7314 - val_loss: 0.4553 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3195 - accuracy: 0.8694 - val_loss: 0.4937 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2197 - accuracy: 0.9181 - val_loss: 0.5607 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1599 - accuracy: 0.9441 - val_loss: 0.6220 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1221 - accuracy: 0.9584 - val_loss: 0.6205 - val_accuracy: 0.7677


In [27]:
model_preds = model_2.predict(val_sentences)
model_preds[:10],model_preds.shape

(array([[0.33325273],
        [0.87741184],
        [0.9980252 ],
        [0.11561754],
        [0.01235959],
        [0.9925639 ],
        [0.6214264 ],
        [0.99813336],
        [0.9982377 ],
        [0.5018107 ]], dtype=float32), (762, 1))

In [28]:
preds = tf.squeeze(tf.round(model_preds))
preds[:10], preds.shape

(<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>,
 TensorShape([762]))

In [29]:
val_sentences[:10], val_labels[:10], preds[:10]

(array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
        'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
        'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
        '@camilacabello97 Internally and externally screaming',
        'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
        'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
        'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
        "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over so

## Model 3 bidirectional lstm

In [30]:
tf.random.set_seed(42)
model_3_embedding = layers.Embedding(input_dim= 10000,
                                     output_dim = 128,
                                     name = "embedding_3")
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.models.Model(inputs, outputs)
model_3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-tra

In [31]:
model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [32]:
model_3.fit(train_sentences, train_labels, epochs=5,
            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.5093 - accuracy: 0.7481 - val_loss: 0.4606 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3135 - accuracy: 0.8708 - val_loss: 0.5144 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2150 - accuracy: 0.9178 - val_loss: 0.5626 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1523 - accuracy: 0.9469 - val_loss: 0.6365 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1083 - accuracy: 0.9639 - val_loss: 0.6509 - val_accuracy: 0.7664


In [33]:
model_3_preds = model_3.predict(val_sentences)
model_3_preds[:10], model_3_preds.shape

(array([[0.04000057],
        [0.8279291 ],
        [0.99842227],
        [0.13531101],
        [0.00311338],
        [0.99220747],
        [0.95528346],
        [0.99945647],
        [0.99898285],
        [0.28141677]], dtype=float32), (762, 1))

In [34]:
preds = tf.squeeze(tf.round(model_3_preds))
preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_sentences[:10],val_labels[:10], preds[:10]

(array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
        'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
        'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
        '@camilacabello97 Internally and externally screaming',
        'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
        'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
        'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
        "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over so

## Model 4 Conv1D

In [42]:
model_4_embedding = layers.Embedding(input_dim=10000,
                                     output_dim = 128,
                                     name = "model_5_embedding")

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.models.Model(inputs, outputs)
model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 model_5_embedding (Embeddin  (None, 15, 128)          1280000   
 g)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 32)            12320     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                           

In [43]:
model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [44]:
model_5_hist = model_4.fit(train_sentences, train_labels, epochs=5,
                           validation_data = (val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.5707 - accuracy: 0.7152 - val_loss: 0.4774 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3541 - accuracy: 0.8532 - val_loss: 0.4701 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2229 - accuracy: 0.9174 - val_loss: 0.5278 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1434 - accuracy: 0.9531 - val_loss: 0.6020 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0994 - accuracy: 0.9661 - val_loss: 0.6720 - val_accuracy: 0.7743


In [45]:
model_4_preds = model_4.predict(val_sentences)
model_4_preds[:10]

array([[0.09579219],
       [0.6171756 ],
       [0.99706334],
       [0.05774188],
       [0.00975666],
       [0.9839972 ],
       [0.96436554],
       [0.9978738 ],
       [0.99752825],
       [0.04596004]], dtype=float32)

In [46]:
preds = tf.squeeze(tf.round(model_4_preds))
val_sentences[:10],val_labels[:10],preds[:10]

(array(['DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
        'FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
        'Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
        '@camilacabello97 Internally and externally screaming',
        'Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
        'Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
        'How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
        "Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over so

## Lets use Pretrained Embeddings using tensroflow hub

### Lets use tensorflow hub

In [51]:
import tensorflow_hub as hub
pretrained_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") 
embed = pretrained_embed(["hello im Amir"])
print(embed)

tf.Tensor(
[[ 1.94125958e-02 -6.03232756e-02  6.93045035e-02  1.17314346e-02
   8.13734978e-02  9.32630524e-03 -5.76505670e-03 -5.71341254e-02
   3.44402120e-02  5.08814910e-03  2.61483341e-02  5.84573410e-02
   1.58521142e-02  7.35401064e-02  3.73358279e-02 -5.81931844e-02
  -8.14725179e-03  2.66613234e-02  7.56410370e-03 -7.89862424e-02
   1.31345773e-03  4.77254158e-03 -2.30624992e-02 -1.53475692e-02
  -3.05198412e-02  1.83395036e-02  4.73744832e-02 -1.96347781e-03
  -3.04156076e-02 -2.06879098e-02  5.22892959e-02  7.70693570e-02
  -3.32761407e-02  6.20163642e-02  1.48938550e-03 -4.77383099e-02
  -3.55370539e-05 -2.19307207e-02 -5.09539992e-02  5.06900065e-02
   6.42933995e-02 -2.96269264e-02  4.45127636e-02  6.29598051e-02
  -6.55748090e-03 -2.37054983e-03 -5.51259844e-03  3.86816524e-02
   2.05194745e-02 -4.80183885e-02 -2.08736658e-02  7.68402591e-03
  -2.30755415e-02  7.15765753e-04 -4.30100970e-02  3.31782885e-02
   5.62695526e-02 -6.69042766e-02  3.58983688e-02  8.02830458e-02